In [178]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
#importo el código como texto
url = "https://sit.goreloslagos.cl/arcgis/rest/services"
soup = BeautifulSoup(requests.get(url).text)

In [5]:
#nombre de carpeta
#extraigo el texto desde <ul> y lo separo
querry = soup.find('div', class_="rbody")
carpeta = querry.ul.text.split('\n')

In [142]:
#url_carpeta
#agrego la url de cada bd (las cuales contiene capas por tematica)
i=1
url_db = []
for i in range(len(carpeta)):
    if carpeta[i] != '' and carpeta[i] != 'System':
        url_db.append(str(url) + '/'+ carpeta[i])

In [157]:
#url_capa
#Agregar url de cada capa a url_capa y el nombre de la capa
url_capa = []
for i in range(len(url_db)):
    soup = BeautifulSoup(requests.get(url_db[i]).text)
    querry = soup.find_all('li')
    for k in range(len(querry)):
        url_capa.append('https://sit.goreloslagos.cl' + querry[k].a['href'] + '/0')
#        nombre_capa.append(('https://sit.goreloslagos.cl' + querry[k].a['href']).replace("/MapServer","").replace(str(url_db[i]),""))
url_capa[11] = url_capa[11].replace("/0","/1")

In [160]:
#Nombre de capa
nombre_capa = []
for i in range(len(url_capa)):
    soup = BeautifulSoup(requests.get(url_capa[i]).text)
    querry = soup.find('div', class_="rbody")
    nombre_capa.append(querry.b.next_sibling)

In [174]:
#Descripcion
descr_capa = []
for i in range(len(url_capa)):
    soup = BeautifulSoup(requests.get(url_capa[i]).text)
    querry = soup.find('div', class_="rbody")
    descripcion = querry.find("b", text="Description: ").next_sibling
    descr_capa.append(descripcion.replace("\n"," "))

In [136]:
#Lista de campos
#se agrega como string cada lista de atributos de capa en una lista mayor
campos = []
campo_ind = []
for i in range(len(url_capa)):
    soup = BeautifulSoup(requests.get(url_capa[i]).text)
    querry = soup.find_all('li')
    campo_ind = []
    for j in range(len(querry)):
        if querry[j].find(text=True).replace("\r\n","") != '\n':
            campo_ind.append(querry[j].find(text=True).replace("\r\n",""))
    campos.append(','.join(campo_ind))

In [202]:
ide_los_lagos = pd.DataFrame(
    {'Nombre_capa': nombre_capa,
     'Url_capa': url_capa,
     'Descripcion': descr_capa,
     'Atributos': campos
    })

In [211]:
ide_los_lagos.to_csv('Info_IDE_Los_Lagos.csv', sep = ";", encoding = "windows-1252")

In [1]:
#Agregar alias de los atributos (no es tan prioritario)
#la ide los lagos tiene buenas descripciones (casi siempre) en los metadatos, es decir fuente, escala.
#la ide subpesca no tiene tan buenas descripciones. la información generada por ellos que es la importante, tiene buenos alias.
#hoy termino esto y mañana veo que onda con los xonos